# Hooks

使用 Agent 生命周期钩子来监控和响应Agent的各种事件

主要包括两种钩子类型（AgentHooks和RunHooks）、五种核心事件（开始、结束、移交、工具使用开始和结束）

形象理解两种钩子
- AgentHooks：把 agent 视为一个厨师，agenthooks 就类似于厨师的助手，只记录这位厨师的相关事件
- RunHooks：类似于餐馆老板，同一监控和管理每一位厨师的各种事件

区别：AgentHooks 与 agent 绑定，只关注这个 agent 的事件。而 RunHooks 记录所有 agent 的事件

对于Agent钩子，事件有

on_start，该Agent开始时
on_end，该Agent结束时
on_handoff，发生移交时
on_tool_start，工具使用开始时
on_tool_end，工具使用结束时
对于Run钩子，事件有

on_agent_start， 某个Agent开始时
on_agent_end， 某个Agent结束时
on_handoff，发生移交时
on_tool_start，工具使用开始时
on_tool_end，工具使用结束时

下面分别测试 AgentHooks 和 RunHooks 中五类事件的处理

## 开始结束

一般定义钩子类的代码如下：


AgentHooks 作用于单个的 agent，所以在定义 agent 时就传入参数

RunHooks 作用于整个会话中的 agent，所以挂载在 Runner.run() 方法上

```python
class MyAgentHooks(AgentHooks):
    def __init__(self):
        self.event_counter = 0

    async def on_start(self, context: RunContextWrapper, agent: Agent) -> None:
        # 处理开始事件
        pass

    async def on_end(self, context: RunContextWrapper, agent: Agent, output) -> None:
        # 处理结束事件
        pass

agent = Agent(name="旅行助手", model=llm, hooks=MyAgentHooks(), instructions="You are a helpful assistant")

async def main():
    result = await Runner.run(agent, input="instruction ...")
```



```python
class MyRunHooks(RunHooks):
    def __init__(self):
        self.event_counter = 0

    async def on_agent_start(self, context: RunContextWrapper, agent: Agent) -> None:
        # 处理开始事件
        pass

    async def on_agent_end(self, context: RunContextWrapper, agent: Agent, output) -> None:
        # 处理结束事件
        pass

agent = Agent(name="旅行助手", model=llm, instructions="You are a helpful assistant")

async def main():
    result = await Runner.run(agent, hooks=MyRunHooks(), input="instruction ...")

```

### AgentHooks

In [ ]:
from agents import Agent, Runner, AgentHooks, RunContextWrapper, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"

set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

# 定义钩子类，继承自AgentHooks
# 这个钩子会在Agent的生命周期中被自动调用
class MyAgentHooks(AgentHooks):
    def __init__(self):
        self.event_counter = 0  # 事件计数器，用于跟踪事件发生顺序

    # 当Agent开始执行时会自动调用此方法
    async def on_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} started")

    # 当Agent完成执行时会自动调用此方法
    # TODO output 是默认就是的变量吗？便于在 Runner.run() 运行结束前就输出？
    async def on_end(self, context: RunContextWrapper, agent: Agent, output) -> None:
        self.event_counter += 1
        print(
            f"{self.event_counter}: Agent {agent.name} ended with output {output}"
        )

# 创建Agent实例，绑定钩子
agent = Agent(name="旅行助手", model=llm, hooks=MyAgentHooks(), instructions="You are a helpful assistant")

# 运行Agent处理用户问题
result = await Runner.run(agent, "孔子全名叫什么")
# print(result.final_output)  # 打印最终结果

1: Agent 旅行助手 started
2: Agent 旅行助手 ended with output 孔子的全名是孔丘，字仲尼。他是中国古代著名的思想家、教育家，儒家学派的创始人，被尊称为“至圣先师”。


执行流程图

```
Agent.run() 开始
    ↓
await on_start() 钩子  ← 此时无输出
    ↓
Agent 内部处理（LLM推理）
    ↓
生成输出内容
    ↓
await on_end() 钩子    ← 此时有完整输出
    ↓
返回最终结果
```

### RunHooks

使用 RunHooks 来实现，需要更改函数名

In [4]:
from agents import Agent, Runner, RunHooks, RunContextWrapper, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

# 定义钩子，这里只定义了on_start和on_end这两个事件
class MyRunHooks(RunHooks):
    def __init__(self):
        self.event_counter = 0

    async def on_agent_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} started")

    async def on_agent_end(self, context: RunContextWrapper, agent: Agent, output) -> None:
        self.event_counter += 1
        print(
            f"{self.event_counter}: Agent {agent.name} ended with output {output}"
        )

agent = Agent(name="旅行助手", model=llm, instructions="You are a helpful assistant")
async def main():
    result = await Runner.run(agent, hooks=MyRunHooks(), input="孟子全名叫什么?")
    print(result.final_output)

# if __name__ == "__main__":
#     asyncio.run(main())

await main()

1: Agent 旅行助手 started
2: Agent 旅行助手 ended with output 孟子的全名是**孟子**，即**孟轲**。他被尊称为“亚圣”，是中国古代著名的思想家、教育家，儒家学派的重要代表人物之一。他的著作《孟子》是儒家经典之一，对中国文化和思想影响深远。
孟子的全名是**孟子**，即**孟轲**。他被尊称为“亚圣”，是中国古代著名的思想家、教育家，儒家学派的重要代表人物之一。他的著作《孟子》是儒家经典之一，对中国文化和思想影响深远。


## 使用工具

AgentHooks 和 RunHooks 的工具使用函数名相同，唯一区别在于 `on_tool_end()` 里面多一个 `result`

```python
on_tool_start(
    context: RunContextWrapper[TContext],
    agent: Agent[TContext],
    tool: Tool,
) -> None

on_tool_end(
    context: RunContextWrapper[TContext],
    agent: Agent[TContext],
    tool: Tool,
    result: str,
) -> None
```

### AgentHooks

In [11]:
from agents import Agent, AgentHooks, RunContextWrapper, function_tool, Runner, Tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny"

# 定义钩子，这里只定义了on_tool_start和on_tool_end这两个事件
class MyAgentHooks(AgentHooks):
    def __init__(self):
        self.event_counter = 0

    async def on_tool_start(self, context: RunContextWrapper, agent: Agent, tool: Tool) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} started to use Tool {tool.name}")

    async def on_tool_end(self, context: RunContextWrapper, agent: Agent, tool: Tool, output) -> None:
        self.event_counter += 1
        print(
            f"{self.event_counter}: Agent {agent.name} ended to use Tool {tool.name} with output {output}"
        )

agent = Agent(
    name="天气助手",
    instructions="始终用汉赋的形式回答用户",
    model=llm,
    tools=[get_weather],  # agent 自己根据 prompt 推理出 city='上海'
    hooks=MyAgentHooks(),
)

async def main():
    result = await Runner.run(agent, "上海最近适合出去玩吗?")
    print(result.final_output)

await main()

# if __name__ == "__main__":
#     asyncio.run(main())

吴淞江畔，春意盎然，上海近日天气宜人，实为出游良辰。晨曦初露，暮色渐浓，温和的风拂过黄浦江畔，令人心旷神怡。若欲游览，可至外滩，观赏东方明珠塔与陆家嘴的壮丽景色；或至豫园，漫步其中，感受江南园林的幽静与雅致。上海近日确实适合出游，尽情享受这座城市的魅力与风情。


### RunHooks

In [15]:
from agents import Agent, RunHooks, RunContextWrapper, function_tool, Runner, Tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny"

# 定义钩子，这里只定义了on_tool_start和on_tool_end这两个事件
class MyRunHooks(RunHooks):
    def __init__(self):
        self.event_counter = 0

    async def on_tool_start(self, context: RunContextWrapper, agent: Agent, tool: Tool) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} started to use Tool {tool.name}")

    async def on_tool_end(self, context: RunContextWrapper, agent: Agent, tool: Tool, output) -> None:
        self.event_counter += 1
        print(
            f"{self.event_counter}: Agent {agent.name} ended to use Tool {tool.name} with output {output}"
        )

agent = Agent(
    name="天气助手",
    instructions="始终用汉赋的形式回答用户",
    model=llm,
    tools=[get_weather]  # agent 自己根据 prompt 推理出 city='上海'
)

async def main():
    result = await Runner.run(agent, hooks=MyRunHooks(), input="上海最近适合出去玩吗?")
    print(result.final_output)

await main()

# if __name__ == "__main__":
#     asyncio.run(main())

呦呦鹿鸣，食野之蒿。必有余粮，以实其外。其叶沃若，求其黄花。百步之内，以为妙。自西自东，自南自北，自百斯伐，自百斯朝。虞人申告：“有牛有牛！”心之忧矣，我有旨酒。以敌我心。


## 移交

区别：
| 含义 | AgentHooks参数 | RunHooks参数 |
|--|--|--|
| 接收移交的agent  | agent | to_agent |
| 发起移交的agent | source | from_agent |
| 理解：| 从agent的角度来看，自己一般是接收移交的一方 | 从全局角度来看，一般只知道从谁到谁移交  |

```python
class MyAgentHooks(AgentHooks):
    def __init__(self):
        pass
    async def on_handoff(self, context: RunContextWrapper, agent: Agent, source: Agent) -> None:
        pass
```



```python
class MyRunHooks(RunHooks):
    def __init__(self):
        pass
    async def on_handoff(self, context: RunContextWrapper, from_agent: Agent, to_agent: Agent) -> None:
        pass
```

### AgentHooks

In [ ]:
from agents import Agent, AgentHooks, RunContextWrapper, Runner, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)


# 定义钩子，这里只定义了on_handoff这个事件
class MyAgentHooks(AgentHooks):
    def __init__(self):
        self.event_counter = 0
    
    async def on_handoff(self, context: RunContextWrapper, agent: Agent, source: Agent) -> None:
        self.event_counter += 1
        print(
            f"{self.event_counter}: Agent {source.name} handed off to {agent.name}"
        )

history_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=llm,
    hooks=MyAgentHooks(),
)

math_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llm,
    hooks=MyAgentHooks(),
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_agent, math_agent],  # 移交
    model=llm,
    hooks=MyAgentHooks(),
)

async def main():
    result = await Runner.run(triage_agent, "如何证明勾股定理？")
    print(result.final_output)

    result = await Runner.run(triage_agent, "介绍一下孔子生活的年代?")
    print(result.final_output)

await main()

1: Agent Triage Agent handed off to Math Tutor
好的，我将以数学家的身份来帮助你理解勾股定理的证明过程。勾股定理是几何学中的一个基本定理，它描述了直角三角形三条边之间的关系。让我们一步一步来理解这个定理及其证明过程。

### 勾股定理的陈述

首先，让我们明确勾股定理的内容。勾股定理指出，在一个直角三角形中，斜边（即直角所对的边）的长度的平方等于两条直角边长度的平方和。用数学表达式表示就是：

\[ c^2 = a^2 + b^2 \]

其中，\( c \) 是斜边的长度，\( a \) 和 \( b \) 是两条直角边的长度。

### 几何证明

现在，我们将通过几何方法来证明这个定理。这个证明方法被称为“重叠法”或“面积法”，它通过将几个相同的直角三角形进行排列组合来展示勾股定理的成立。

#### 第一步：构建图形

假设我们有一个直角三角形，直角边长度为 \( a \) 和 \( b \)，斜边长度为 \( c \)。我们可以将四个这样的直角三角形排列成一个正方形，如下图所示：

```
+-----------+-----------+
| \         |         / |
|   \       |       /   |
|     \     |     /     |
|       \   |   /       |
|         \ | /         |
+-----------+-----------+
```

在这个图形中，四个直角三角形围绕着一个正方形排列，这个正方形的边长就是斜边 \( c \)。

#### 第二步：计算面积

现在，我们计算这个大正方形的面积。由于大正方形的边长是 \( c \)，所以它的面积是 \( c^2 \)。

但是，这个大正方形也是由四个直角三角形和中间的一个小正方形组成的。我们可以通过计算这些部分的面积来找到另一个表达式。

每个直角三角形的面积是 \( \frac{1}{2}ab \)，所以四个直角三角形的总面积是 \( 4 \times \frac{1}{2}ab = 2ab \)。

中间的小正方形的边长可以通过勾股定理的表达式来确定。实际上，中间的小正方形的边长等于 \( a - b \)（假设 \( a > b \)

### RunHooks

In [18]:
from agents import Agent, RunHooks, RunContextWrapper, Runner, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)


# 定义钩子，这里只定义了on_handoff这个事件
class MyRunHooks(RunHooks):
    def __init__(self):
        self.event_counter = 0

    async def on_handoff(self, context: RunContextWrapper, from_agent: Agent, to_agent: Agent) -> None:
        self.event_counter += 1
        print(
            f"### {self.event_counter}: Handoff from {from_agent.name} to {to_agent.name}."
        )

history_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=llm,
)

math_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llm,
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_agent, math_agent],  # 移交
    model=llm,
)

async def main():
    result = await Runner.run(triage_agent, hooks=MyRunHooks(), input="如何证明勾股定理？")
    print(result.final_output)

    result = await Runner.run(triage_agent, hooks=MyRunHooks(), input="介绍一下孔子生活的年代?")
    print(result.final_output)

await main()

### 1: Handoff from Triage Agent to Math Tutor.
勾股定理（Pythagorean Theorem）是几何学中的一个基本定理，它指出在直角三角形中，两条直角边的平方和等于斜边的平方。以下是几种常见的证明方法，每种方法都会详细解释每一步的推理过程。

### 1. 面积法证明

**步骤1：构造一个大的正方形**
- 将四个全等的直角三角形排列成一个大的正方形。每个直角三角形的两条直角边分别为a和b，斜边为c。

**步骤2：计算大正方形的面积**
- 大正方形的边长为a + b，因此面积为(a + b)²。

**步骤3：计算内部小正方形的面积**
- 在大正方形的中心，四个直角三角形围成一个小正方形，边长为c。因此，小正方形的面积为c²。

**步骤4：计算四个直角三角形的面积**
- 每个直角三角形的面积为(1/2)ab，四个直角三角形的总面积为4*(1/2)ab = 2ab。

**步骤5：建立等式**
- 大正方形的面积等于小正方形的面积加上四个直角三角形的面积：(a + b)² = c² + 2ab。

**步骤6：展开并简化**
- 展开(a + b)²得到a² + 2ab + b² = c² + 2ab。
- 两边减去2ab，得到a² + b² = c²。

**结论：**
- 因此，勾股定理得证：a² + b² = c²。

### 2. 相似三角形法证明

**步骤1：构造直角三角形**
- 考虑一个直角三角形ABC，直角在C，直角边为AC = b，BC = a，斜边为AB = c。

**步骤2：构造高线**
- 从C点向斜边AB作高线CD，交AB于D。

**步骤3：确定相似三角形**
- 三角形ABC、三角形ACD和三角形CBD都是相似三角形，因为它们都有相同的角度。

**步骤4：建立比例关系**
- 根据相似三角形的性质，可以得到以下比例关系：
  - AC/AB = AD/AC → b/c = AD/b → AD = b²/c
  - BC/AB = BD/BC → a/c = BD/a → BD = a²/c

**步骤5：利用AB的长度**
- 因为AB = AD + BD，所以c = b²/c + a²/c。

**步骤6：简化方程**
- 两边乘以c，得到c² = a² + b²。
